In [15]:
from IPython.display import clear_output

# make sure your kaggle api "kaggle.json" file in your drive
!mkdir -p /root/.kaggle
!echo '{"username":"tarobxl","key":"a4b1e63bbff7f44f713dce2525191ba1"}' > /root/.kaggle/kaggle.json
#! cp '/content/drive/My Drive/kaggle.json' /root/.kaggle # <---- path for kaggle.json file
!chmod 400 /root/.kaggle/kaggle.json
!cat /root/.kaggle/kaggle.json

!pip uninstall -y kaggle >> quit
!pip install --upgrade pip >> quit
!pip install kaggle==1.5.6 >> quit
!kaggle -v >> quit

clear_output()

In [16]:
filenames = ['../input/k200-tf-0-20210727-122841/data_features.pickle',
 '../input/k200-tf-1-20210727-123059/data_features.pickle',
 '../input/k200-tf-2-20210727-144220/data_features.pickle',
 '../input/k200-tf-3-20210727-170129/data_features.pickle',
 '../input/k200-tf-4-20210727-200848/data_features.pickle',
 '../input/k200-tf-5-20210728-174514/data_features.pickle',
 '../input/k200-tf-6-20210728-114118/data_features.pickle',
 '../input/k200-tf-7-20210728-115026/data_features.pickle',
 '../input/k200-tf-8-20210728-113919/data_features.pickle',
 '../input/k200-tf-9-20210728-113846/data_features.pickle',
 '../input/k200-tf-query-images-20210728-154427/data_features.pickle']

datasets = [f.split("/")[2] for f in filenames]
datasets

['k200-tf-0-20210727-122841',
 'k200-tf-1-20210727-123059',
 'k200-tf-2-20210727-144220',
 'k200-tf-3-20210727-170129',
 'k200-tf-4-20210727-200848',
 'k200-tf-5-20210728-174514',
 'k200-tf-6-20210728-114118',
 'k200-tf-7-20210728-115026',
 'k200-tf-8-20210728-113919',
 'k200-tf-9-20210728-113846',
 'k200-tf-query-images-20210728-154427']

In [ ]:
!mkdir -p features
!rm -rf features/*
!rm -rf *.pickle
!rm -rf *.zip
for dataset in datasets:
    url_suffix = f"tarobxl/{dataset}"
    !kaggle datasets download $url_suffix -p /content/ --unzip
    !mkdir -p features/$dataset
    !mv data_features.pickle features/$dataset/

 99% 722M/729M [00:04<00:00, 147MB/s]
100% 729M/729M [00:04<00:00, 155MB/s]
100% 727M/729M [00:04<00:00, 169MB/s]
100% 729M/729M [00:04<00:00, 182MB/s]
100% 729M/729M [00:04<00:00, 101MB/s] 
100% 729M/729M [00:04<00:00, 164MB/s]
 99% 719M/729M [00:07<00:00, 129MB/s]
100% 729M/729M [00:07<00:00, 100MB/s]
 98% 714M/729M [00:04<00:00, 201MB/s]
100% 729M/729M [00:04<00:00, 158MB/s]
100% 728M/729M [00:06<00:00, 109MB/s] 
100% 729M/729M [00:06<00:00, 116MB/s]
 99% 724M/729M [00:03<00:00, 176MB/s]
100% 729M/729M [00:03<00:00, 199MB/s]


In [ ]:
import pandas as pd
import numpy as np
import glob
import pickle
import os
import gc

import h5py
from tqdm import tqdm

query_filename = datasets[-1]

ids = []
embeddings = []
for dataset in tqdm(datasets):
    filename = f"features/{dataset}/data_features.pickle"
    if "query" in filename:
        query_filename = filename
        print(query_filename)
        continue
    file_to_read = open(filename, "rb")
    loaded_features = pickle.load(file_to_read)
    file_to_read.close()
    
    ids += loaded_features["ids"]
    embeddings += loaded_features["embeddings"]
    
print(len(ids), len(embeddings))
ids[0], embeddings[0]

In [ ]:
ids, embeddings = zip(*sorted(zip(ids, embeddings)))
print(ids[95763], embeddings[95763])

X = np.vstack(embeddings)
X.shape

In [ ]:
%%capture
if False:
    !pip install pca
    from pca import pca
    #nb_dim = 64 # max 256
    #model = pca(n_components=nb_dim, normalize=True)
    #results = model.fit_transform(X)

In [ ]:
if False:
    from sklearn.decomposition import IncrementalPCA
    transformer = IncrementalPCA(n_components=64, batch_size=200)
    transformer.partial_fit(X[:1*1000, :])
    X_transformed = transformer.fit_transform(X)
    X_transformed.shape

In [ ]:
from sklearn import decomposition

n_comp = 64
svd = decomposition.TruncatedSVD(n_components=n_comp, algorithm='arpack')
svd.fit(X[:100*1000, :])
print(svd.explained_variance_ratio_.sum())

X_transformed = svd.transform(X)
X_transformed.shape

In [ ]:
file_to_read = open(query_filename, "rb")
loaded_features = pickle.load(file_to_read)
file_to_read.close()

query_ids = loaded_features["ids"]
query_embeddings = loaded_features["embeddings"]

print(query_ids[0], query_embeddings[0])

query_ids, query_embeddings = zip(*sorted(zip(query_ids, query_embeddings)))
print(query_ids[38391], query_embeddings[38391])

query_X = np.vstack(query_embeddings)
query_X.shape

In [ ]:
query_X_transformed = svd.transform(query_X)
query_X_transformed.shape

In [ ]:
import h5py
import numpy as np

M_ref = X_transformed #np.random.rand(1_000_000, 256).astype('float32')
M_query = query_X_transformed #np.random.rand(50_000, 256).astype('float32')
print(M_ref.shape, M_query.shape)

qry_ids = ['Q' + str(x).zfill(5) for x in range(50_000)]
ref_ids = ['R' + str(x).zfill(6) for x in range(1_000_000)]

out = "fb-isc-submission.h5"
with h5py.File(out, "w") as f:
    f.create_dataset("query", data=M_query)
    f.create_dataset("reference", data=M_ref)
    f.create_dataset('query_ids', data=qry_ids)
    f.create_dataset('reference_ids', data=ref_ids)

In [ ]:
!ls -alh

In [ ]:
!curl --upload-file ./fb-isc-submission.h5 https://transfer.sh/fb-isc-submission.h5

In [ ]:
from google.colab import files
files.download('fb-isc-submission.h5')